First, we need to import several packages to use the APIs.

In [8]:
import pandas as pd

pd.read_csv('Louisville_Metro_KY_-_Employee_Salary_Data.csv', usecols=[1,3,4], )

,Employee_Name,jobTitle,Annual_Rate
0,"Martin, Kyle D",Fire Battalion Chief 56hr,104089.85
1,"Summers, William E",Board Member,3939.00
2,"Gibson, Osburne B",Director,146946.80
3,"Addison, Joseph Lee",Library Assistant LU,45427.20
4,"Martin, David",Library Page LU,29577.60
...,...,...,...
40825,"Brooks, Troya",Youth Program Worker U310,33966.40
40826,"McKee, Joshua",Workforce Solutions Manager,63000.60
40827,"Sands, Shelton",Sanitation Tipper,35339.20
40828,"Ware, Meghan",Victim Advocate,34008.00
